SQL query:

Create an ordered list with the city of each province with the most monuments, along with the counts of the monuments. 
   
Based on Hanson, J. W. (2016) Cities Database (OXREP databases) DOI: https://doi.org/10.5287/bodleian:eqapevAn8

In [1]:
import pandas as pd
import sqlite3

# use pandas to create count of monuments for each location

In [2]:
# first, I'm going to add a new 'monumentcount' column to the dataframes. 

# Read the excel as pandas DataFrame, first for the cities sheet then for the monuments sheet
cities_df = pd.read_excel('Hanson2016_CitiesDatabase_OxREP.xlsx', sheet_name='cities')
monuments_df = pd.read_excel('Hanson2016_CitiesDatabase_OxREP.xlsx', sheet_name='monuments')

In [3]:
cities_df.head(1)

,id,ancient_toponym,modern_toponym,province,country,barrington_atlas_rank,barrington_atlas_reference,start_date,end_date,longitude,latitude,select_bibliography
0,Hanson2016_1,Abae,Kalapodi,Achaea,Greece,4 or 5,55 D3,-600,NaN,22.933333,38.583333,BNP; Hansen 2006; Hansen and Nielsen 2004; PEC...


In [4]:
# # this counts how many times each id in the cities sheet shows up in the monuments sheet
# for ids in cities_df['id']:
#     print(f"{ids}: {monuments_df[(monuments_df['id'] == ids)].shape[0]}")
    
# run this to check answers below

In [5]:
# turns the above into a dictionary of ids and monument counts
MonCount = {}
for ids in cities_df['id']:
    MonCount.update({ids: monuments_df[(monuments_df['id'] == ids)].shape[0]}) 

In [6]:
# first, add a new column to the cities sheet with a 'monument count' variable. 
# Then, set the new column's values with the map function and the dictionary we made above
cities_df['MonumentCount'] = cities_df['id'].map(MonCount)

In [7]:
# # tada! now we have a monument count value
cities_df[['id','MonumentCount']]

,id,MonumentCount
0,Hanson2016_1,5
1,Hanson2016_2,3
2,Hanson2016_3,7
3,Hanson2016_4,8
4,Hanson2016_5,8
...,...,...
1383,Hanson2016_1384,11
1384,Hanson2016_1385,12
1385,Hanson2016_1386,6
1386,Hanson2016_1387,9


a different way to do this, by grouping the monument sheet rather than counting thru it. probably easier; same result

In [8]:
# # This line resets the memory.
# connection = sqlite3.connect(":memory:")

# # Read the excel as pandas DataFrame, first for the cities sheet then for the monuments sheet
# cities_df = pd.read_excel('Hanson2016_CitiesDatabase_OxREP.xlsx', sheet_name='cities')
# monuments_df = pd.read_excel('Hanson2016_CitiesDatabase_OxREP.xlsx', sheet_name='monuments')


In [9]:
# # groups monument sheet by id
# monsgrouped = monuments_df.groupby("id")

In [10]:
# gets the count of monuments for each id
# monsgrouped['structure'].count()

In [11]:
# # assigns a new column in cities_df based on the monument counts
# cities_df['MonumentCount'] = cities_df['id'].map(monsgrouped['structure'].count())

In [12]:
# # tada! now we have a monument count value
# cities_df[['id','MonumentCount']]

.

In [13]:
# sorting by mon count
cities_df[['id','ancient_toponym','province','MonumentCount']].sort_values('MonumentCount', ascending=False)

,id,ancient_toponym,province,MonumentCount
991,Hanson2016_992,Roma,Italia (VII Etruria),236
13,Hanson2016_14,Athenae,Achaea,107
25,Hanson2016_26,Corinthia,Achaea,56
304,Hanson2016_305,Pergamum,Asia,50
179,Hanson2016_180,Lepcis Magna,Africa Proconsularis,46
...,...,...,...,...
539,Hanson2016_540,Novae (Dalmatia),Dalmatia,0
198,Hanson2016_199,Tacapae,Africa Proconsularis,0
915,Hanson2016_916,Pausulae,Italia (V Picenum),0
914,Hanson2016_915,Numana,Italia (V Picenum),0


# Back to sql

In [15]:
# # This line resets the memory.
connection = sqlite3.connect(":memory:")


# Turn the DataFrames to SQLs
cities_df.to_sql('cities', con = connection, index = False)
monuments_df.to_sql('mons', con = connection, index = False)

9471

In [42]:
pd.read_sql("""
SELECT id, province, MonumentCount
FROM cities
ORDER BY MonumentCount DESC
""", connection)

,id,province,MonumentCount
0,Hanson2016_992,Italia (VII Etruria),236
1,Hanson2016_14,Achaea,107
2,Hanson2016_26,Achaea,56
3,Hanson2016_305,Asia,50
4,Hanson2016_180,Africa Proconsularis,46
...,...,...,...
1383,Hanson2016_1357,Syria Palestina,0
1384,Hanson2016_1360,Syria Palestina,0
1385,Hanson2016_1372,Thracia,0
1386,Hanson2016_1377,Thracia,0


In [57]:
pd.read_sql("""
SELECT id as id, ancient_toponym as name, province AS province,
MAX(MonumentCount) AS topmons
FROM cities
GROUP BY province
ORDER BY topmons DESC
""", connection)

,id,name,province,topmons
0,Hanson2016_992,Roma,Italia (VII Etruria),236
1,Hanson2016_14,Athenae,Achaea,107
2,Hanson2016_305,Pergamum,Asia,50
3,Hanson2016_180,Lepcis Magna,Africa Proconsularis,46
4,Hanson2016_1258,Thamugadi,Numidia,40
5,Hanson2016_481,Cyrene,Creta et Cyrenaica,38
6,Hanson2016_788,Ostia,Italia (I Latium and Campania),37
7,Hanson2016_1375,Byzantium,Thracia,36
8,Hanson2016_1324,Antiochia (Syria),Syria,33
9,Hanson2016_658,Vienna,Gallia Narbonensis,32
